In [102]:
import numpy as np 
import pandas as pd 
import re
import matplotlib.pyplot as plt 
import sklearn.metrics as mt 
from sklearn.tree import export_graphviz 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier ## check
from sklearn.metrics import accuracy_score      ##check
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedShuffleSplit  ## check
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV 
import joblib 
from sklearn.metrics import confusion_matrix

###sdjflkdsjlskdf

In [63]:
 # 1. daily raw data 가져오기
df = pd.read_excel(r"C:\Users\siim2\OneDrive\바탕 화면\sic 펀드\4주차\auto_raw_new.xlsx", sheet_name="Sheet1", header=0, index_col=0)

In [64]:
df.head(5)

,auto,KOSPI지수,원/달러 환율,원/엔 환율,국채 3년물,국채 10년물,PER,PBR,VKOSPI,treasury_10_3_diff,...,us_e_infla_60,yield_gap(%)_60,cpi_60,export_60,WTI_60,DGS2_60,DGS10_60,TIPS_60,VIX_60,PPI_60
2010-03-31,up,1692.85,1131.3,1213.45,3.895,4.94,24.35,1.36,13.47,1.06,...,-0.001520,0.041108,0.001143,-0.003024,0.000843,0.001338,0.000067,0.002583,-0.003504,0.000217
2010-04-01,up,1719.17,1126.4,1204.45,3.895,4.92,24.13,1.38,13.63,1.07,...,-0.001448,0.038560,0.000667,-0.003721,0.001020,0.001886,0.000202,0.002811,-0.003742,0.000212
2010-04-05,up,1724.99,1123.1,1188.97,3.840,4.95,24.22,1.38,13.56,1.11,...,-0.001381,0.036369,0.000195,-0.004392,0.001237,0.002512,0.000351,0.003084,-0.004067,0.000207
2010-04-06,up,1726.09,1123.2,1195.40,3.845,4.94,24.24,1.38,14.75,1.10,...,-0.001299,0.033592,-0.000272,-0.005038,0.001483,0.003055,0.000521,0.003383,-0.004470,0.000200
2010-04-07,up,1726.60,1120.5,1191.13,3.820,4.88,24.26,1.39,14.22,1.06,...,-0.001157,0.031333,-0.000736,-0.005658,0.001698,0.003488,0.000649,0.003483,-0.004842,0.000193


In [65]:
print("facter type:\n %s " % df.columns)
print("갯수 : %i " %len(df.columns))

facter type:
 Index(['auto', 'KOSPI지수', '원/달러 환율', '원/엔 환율', '국채 3년물 ', '국채 10년물 ', 'PER',
       'PBR', 'VKOSPI', 'treasury_10_3_diff', 'T10Y2Y', 'usa_cpi',
       'us_e_infla', 'yield_gap(%)', 'cpi', 'export', 'WTI', 'DGS2', 'DGS10',
       'TIPS', 'VIX', 'PPI', 'USA_Vehi_Sell', 'KOSPI지수_20', '원/달러 환율_20',
       '원/엔 환율_20', '국채 3년물_20', '국채 10년물_20', 'PER_20', 'PBR_20', 'VKOSPI_20',
       'treasury_10_3_diff_20', 'T10Y2Y_20', 'usa_cpi_20', 'us_e_infla_20',
       'yield_gap(%)_20', 'cpi_20', 'export_20', 'WTI_20', 'DGS2_20',
       'DGS10_20', 'TIPS_20', 'VIX_20', 'PPI_20', 'KOSPI지수_60', '원/달러 환율_60',
       '원/엔 환율_60', '국채 3년물 _60', '국채 10년물 _60', 'PER_60', 'PBR_60',
       'VKOSPI_60', 'treasury_10_3_diff_60', 'T10Y2Y_60', 'usa_cpi_60',
       'us_e_infla_60', 'yield_gap(%)_60', 'cpi_60', 'export_60', 'WTI_60',
       'DGS2_60', 'DGS10_60', 'TIPS_60', 'VIX_60', 'PPI_60'],
      dtype='object') 
갯수 : 65 


In [104]:
# 2. features, label 전체데이터 생성
# X, y는 최근일까지 포함한 전 데이터. X_past, y_past는 결과가 확인된 61일전까지의 데이터
# 결측치가 있는 행 제거
df_dropna = df.dropna()

if df_dropna.empty:
    print("error")
else:
    # 종속 변수(y) 및 독립 변수(X) 설정
    y = df_dropna['auto']
    X = df_dropna.drop(columns=['auto'])

    # 최적화된 코드를 사용하여 y_past 및 X_past 생성
    # 위에서 이미 결측치를 제거하였으므로 추가적인 처리가 필요하지 않습니다.
    y_past = y
    X_past = X

# 3. train, test 나누기

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

for train_index, test_index in sss.split(X_past, y_past): #sss.split(~) 안에 n_splits 수만큼 준비됨
    X_train, X_test = X_past.iloc[train_index,], X_past.iloc[test_index,] 
    y_train, y_test = y_past[train_index], y_past[test_index]
# 훈련 데이터와 테스트 데이터의 클래스 비율 확인
train_class_counts = {label: sum(y_train == label) for label in set(y)}
test_class_counts = {label: sum(y_test == label) for label in set(y)}

print("훈련 데이터 클래스 비율:", train_class_counts)
print("테스트 데이터 클래스 비율:", test_class_counts)
print()
 

훈련 데이터 클래스 비율: {'up': 838, 'neutral': 1013, 'down': 755}
테스트 데이터 클래스 비율: {'up': 210, 'neutral': 253, 'down': 189}



C:\Users\siim2\AppData\Local\Temp\ipykernel_16008\3841303264.py:24: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y_train, y_test = y_past[train_index], y_past[test_index]


In [84]:
# 4. 모델 세부 튜닝: 최적 하이퍼파라미터 찾기
rnd_clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0) 
 
param_dist_rf = { 
 'n_estimators':[50, 100, 500], 
 'max_leaf_nodes':[20, 30, 40, 50], 
 'max_features':[2, 4, 6, 8] 
} 

rnd_search = RandomizedSearchCV(rnd_clf, param_dist_rf, cv=10, random_state=42) 
rnd_search.fit(X_train, y_train) 
print(rnd_search.best_params_)

{'n_estimators': 100, 'max_leaf_nodes': 40, 'max_features': 8}


In [68]:
# 5. 학습 및 K-fold cross_validation 평가 
rnd_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=40, max_features=8, n_jobs=-1, 
random_state=42) #디폴트
rnd_scores = cross_val_score(rnd_clf, X_train, y_train, scoring="accuracy", cv=10) 
print("\n<10-fold cross-validation>")
print("accuracy score mean: ", rnd_scores.mean())


<10-fold cross-validation>
accuracy score mean:  0.7416666666666668


In [69]:
# 6. 최종 모델 학습
rnd_clf.fit(X_train, y_train) 
print("\n<AI model: machine learning done >") 
print("accuracy_score of train data(0.8 of sample): ", rnd_clf.score(X_train, y_train))


<AI model: machine learning done >
accuracy_score of train data(0.8 of sample):  0.8271929824561404


In [70]:
# 7. test data 확인
print("accuracy_score of test data(0.2 of sample): ", rnd_clf.score(X_test, y_test)) 
y_test_pred = rnd_clf.predict(X_test) 
print("accuracy_score of test data: ", mt.accuracy_score(y_test, y_test_pred)) 

accuracy_score of test data(0.2 of sample):  0.7300613496932515
accuracy_score of test data:  0.7300613496932515


In [75]:
# 8. confusion matrix 확인
y_test_pred = rnd_clf.predict(X_test) 
cm1= confusion_matrix(y_test, y_test_pred, labels=["up","neutral","down"]) 
print("\n<Confusion matrix>") 
print("(of test)") 
print("up","neutral","down") 
print(cm1) 

cm2= confusion_matrix(y_past, rnd_clf.predict(X_past), labels=["up","neutral","down"]) 
print("(of all)") 
print("up","neutral","down") 
print(cm2) 



<Confusion matrix>
(of test)
up neutral down
[[239  75   1]
 [ 61 281  38]
 [ 17  72 194]]
(of all)
up neutral down
[[ 857  189    2]
 [ 151 1030   85]
 [  42  189  713]]


In [76]:
# 9. 변수 중요도 체크
print("\n<Feature importance>") 
for name, score in zip(X.columns, rnd_clf.feature_importances_): 
 print(name, ": ", score)


<Feature importance>
KOSPI지수 :  0.03209680315560768
원/달러 환율 :  0.0155572148607508
원/엔 환율 :  0.019401599851591785
국채 3년물  :  0.02131544736432627
국채 10년물  :  0.009273193126079793
PER :  0.02523488782587946
PBR :  0.006936999253397929
VKOSPI :  0.011206619464298552
treasury_10_3_diff :  0.021042462404111336
T10Y2Y :  0.013493663418147905
usa_cpi :  0.014187639875596703
us_e_infla :  0.013007463848311786
yield_gap(%) :  0.027369496901965936
cpi :  0.015130323591023547
export :  0.019349787346407455
WTI :  0.015943917006954217
DGS2 :  0.012675842727075257
DGS10 :  0.01704542658316859
TIPS :  0.013113279380845428
VIX :  0.03022213693514989
PPI :  0.017840115545734496
USA_Vehi_Sell :  0.024003034380701532
KOSPI지수_20 :  0.01679596547079583
원/달러 환율_20 :  0.014807357399179885
원/엔 환율_20 :  0.014408619672920564
국채 3년물_20 :  0.010964271725353842
국채 10년물_20 :  0.013334397975951803
PER_20 :  0.013569742734940738
PBR_20 :  0.010850126243808308
VKOSPI_20 :  0.011218795143306115
treasury_10_3_diff_20 :

In [77]:
# 10. backtesting용 과거의 예측데이터 생성
y_prediction = rnd_clf.predict(X) 
y_pred = pd.Series(y_prediction, index=y.index) 


In [78]:
# 11. 모델 저장
joblib.dump(rnd_clf, "auto_model.pkl")
print("\n< AI model: save >")


< AI model: save >
